# Pandas

##### 1. Загрузить данные в pandas
##### 4. Средствами pandasa соедините данные по оценкам с данными по пользователям (u.user)

In [15]:
import pandas as pd
import numpy as np

In [2]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']

In [3]:
users=pd.read_csv('u.user.csv',sep='|',names=u_cols)
ratings = pd.read_csv('u.data.csv',sep='\t', names=r_cols)
movies = pd.read_csv('u.item.csv', sep='|', names=m_cols, encoding='latin-1', usecols=range(5))
df_all=pd.merge(users,ratings)
df_all=pd.merge(df_all,movies)
df_all.head()

,user_id,age,sex,occupation,zip_code,movie_id,rating,timestamp,title,release_date,video_release_date,imdb_url
0,1,24,M,technician,85711,61,4,878542420,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...
1,13,47,M,educator,29206,61,4,882140552,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...
2,18,35,F,other,37212,61,4,880130803,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...
3,58,27,M,programmer,52246,61,5,884305271,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...
4,59,49,M,educator,08403,61,4,888204597,Three Colors: White (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Trzy%20kolory...


##### 2. Посчитайте среднюю оценку, которую поставил каждый пользователь

In [4]:
avg_rating = df_all[['user_id','rating',]].groupby('user_id',as_index=False).mean().head()
avg_rating.rename(columns={'rating':'avg_rating'}, inplace=True)
avg_rating.head()

,user_id,avg_rating
0,1,3.610294
1,2,3.709677
2,3,2.796296
3,4,4.333333
4,5,2.874286


##### 3. Отсортируйте фильмы по количеству оценок

In [5]:
df_all.groupby('title').count().reset_index().sort_values('rating', ascending=False).head()

,title,user_id,age,sex,occupation,zip_code,movie_id,rating,timestamp,release_date,video_release_date,imdb_url
1398,Star Wars (1977),583,583,583,583,583,583,583,583,583,0,583
333,Contact (1997),509,509,509,509,509,509,509,509,509,0,509
498,Fargo (1996),508,508,508,508,508,508,508,508,508,0,508
1234,Return of the Jedi (1983),507,507,507,507,507,507,507,507,507,0,507
860,Liar Liar (1997),485,485,485,485,485,485,485,485,485,0,485


или 

In [7]:
df_all['title'].value_counts().head()

Star Wars (1977)             583
Contact (1997)               509
Fargo (1996)                 508
Return of the Jedi (1983)    507
Liar Liar (1997)             485
Name: title, dtype: int64

##### 5. Построить модель, которая предсказывает оценку для фильма и пользователя 

In [8]:
categorical_subset = df_all[['sex', 'occupation', 'title']]
categorical_subset = pd.get_dummies(categorical_subset)

In [9]:
df_all_cut = df_all.drop(['zip_code', 'video_release_date', 'imdb_url', 'release_date', 'sex', 'occupation', 'title'], axis=1)

In [10]:
df_mod = pd.concat([df_all_cut, categorical_subset], axis = 1)
df_mod.head()

,user_id,age,movie_id,rating,timestamp,sex_F,sex_M,occupation_administrator,occupation_artist,occupation_doctor,...,title_Yankee Zulu (1994),title_Year of the Horse (1997),title_You So Crazy (1994),title_Young Frankenstein (1974),title_Young Guns (1988),title_Young Guns II (1990),"title_Young Poisoner's Handbook, The (1995)",title_Zeus and Roxanne (1997),title_unknown,title_Á köldum klaka (Cold Fever) (1994)
0,1,24,61,4,878542420,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,13,47,61,4,882140552,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,35,61,4,880130803,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,58,27,61,5,884305271,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,59,49,61,4,888204597,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = df_mod.drop(['rating'], axis=1)
y = df_mod.rating

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

##### 6. Оценить качество регрессора на отложенной выборке по одной из метрик на выбор - RMSE, RMAE, R^2

In [16]:
from sklearn import metrics
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Root Mean Squared Error: 0.9870689270089839


# Spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [ ]:
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

##### 7. Загрузить данные в spark

In [ ]:
df_data = spark.read.csv('u.data.csv', sep = '\t').toDF('user_id', 'movie_id', 'rating', 'timestamp')
df_data.show()

In [ ]:
df_movies = spark.read.csv('u.item.csv', sep='|')
df_movies = df_movies.drop('_c5', '_c6', '_c7', '_c8', '_c9', '_c10', '_c11', 
                           '_c12', '_c13', '_c14', '_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c22', '_c23').toDF('movie_id', 
                                                                                                                                'title', 'release_date', 'video_release_date', 'imdb_url')

df_movies.show()

In [ ]:
df_all = df_data.join(df_movies, df_data.movie_id == df_movies.movie_id)
df_all.show()

##### 8. Средствами спарка вывести среднюю оценку для каждого фильма

In [ ]:
from pyspark.sql.functions import col, avg as avg_
df_all.groupBy('title').agg(avg_('rating')).show()

##### 9. В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами

Топ-5 популярных

In [ ]:
Top_5pop = df_all.groupby('title').count().sort('count', ascending=False).show(5)

Топ-5 непопулярных

In [ ]:
Top_5unpop = df_all.groupby('title').count().sort('count', ascending=True).show(5)